# 01 - Esplorazione Dataset Industrial IoT

Questo notebook esplora il dataset dei dispositivi IoT industriali per comprendere:
- Struttura e caratteristiche dei dati
- Distribuzione delle variabili
- Correlazioni tra features
- Identificazione di outliers e missing values
- Analisi per tipo di dispositivo

In [ ]:
# Import delle librerie necessarie
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
import yaml
import os

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configurazione per i grafici
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

In [ ]:
# Caricamento configurazione
with open('../config.yaml', 'r') as file:
    config = yaml.safe_load(file)

print("Configurazione caricata:")
print(f"- Dataset path: {config['data']['raw_data_path']}")
print(f"- Algoritmo selezionato: {config['model']['algorithm']}")

In [ ]:
# Caricamento del dataset
data_path = config['data']['raw_data_path']
df = pd.read_csv(data_path)

print(f"Dataset caricato con successo!")
print(f"Forma del dataset: {df.shape}")
print(f"Memoria utilizzata: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## 1. Analisi Strutturale del Dataset

In [ ]:
# Informazioni generali sul dataset
print("=== INFORMAZIONI DATASET ===")
print(df.info())
print("\n" + "="*50)

# Prime righe del dataset
print("\n=== PRIME 5 RIGHE ===")
display(df.head())

In [ ]:
# Analisi delle colonne
print("=== ANALISI COLONNE ===")
print(f"Numero totale di colonne: {len(df.columns)}")
print(f"\nColonne numeriche: {df.select_dtypes(include=[np.number]).columns.tolist()}")
print(f"\nColonne categoriche: {df.select_dtypes(include=['object']).columns.tolist()}")

# Tipi di dati
print("\n=== TIPI DI DATI ===")
for col in df.columns:
    print(f"{col}: {df[col].dtype} - Valori unici: {df[col].nunique()}")

In [ ]:
# Analisi dei valori mancanti
print("=== VALORI MANCANTI ===")
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_df = pd.DataFrame({
    'Colonna': missing_data.index,
    'Valori Mancanti': missing_data.values,
    'Percentuale': missing_percent.values
})

missing_df = missing_df[missing_df['Valori Mancanti'] > 0].sort_values('Valori Mancanti', ascending=False)

if len(missing_df) > 0:
    display(missing_df)
    
    # Visualizzazione valori mancanti
    plt.figure(figsize=(12, 6))
    sns.barplot(data=missing_df, x='Colonna', y='Percentuale')
    plt.title('Percentuale di Valori Mancanti per Colonna')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("✅ Nessun valore mancante trovato!")

## 2. Analisi dei Tipi di Dispositivi

In [ ]:
# Distribuzione dei tipi di macchine
print("=== DISTRIBUZIONE TIPI DI DISPOSITIVI ===")
machine_counts = df['Machine_Type'].value_counts()
print(f"Numero di tipi di dispositivi diversi: {df['Machine_Type'].nunique()}")
print("\nTop 10 dispositivi più comuni:")
display(machine_counts.head(10))

# Visualizzazione distribuzione
plt.figure(figsize=(15, 8))
plt.subplot(1, 2, 1)
machine_counts.head(15).plot(kind='bar')
plt.title('Top 15 Tipi di Dispositivi')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
plt.pie(machine_counts.head(10).values, labels=machine_counts.head(10).index, autopct='%1.1f%%')
plt.title('Top 10 Dispositivi - Distribuzione Percentuale')

plt.tight_layout()
plt.show()

In [ ]:
# Classificazione dispositivi secondo le specifiche del progetto
devices_common_only = ['3D_Printer', 'AGV', 'Automated_Screwdriver', 'CMM', 'Carton_Former', 'Compressor', 'Conveyor_Belt', 'Crane', 'Dryer', 'Forklift_Electric', 'Grinder', 'Labeler', 'Mixer', 'Palletizer', 'Pick_and_Place', 'Press_Brake', 'Pump', 'Robot_Arm', 'Shrink_Wrapper', 'Shuttle_System', 'Vacuum_Packer', 'Valve_Controller', 'Vision_System', 'XRay_Inspector']

# Dispositivi con caratteristiche aggiuntive
laser_devices = ['Laser_Cutter']
hydraulic_devices = ['Hydraulic_Press', 'Injection_Molder']
coolant_devices = ['CNC_Lathe', 'CNC_Mill', 'Industrial_Chiller']
heat_devices = ['Boiler', 'Furnace', 'Heat_Exchanger']

# Classificazione
def classify_device(machine_type):
    if machine_type in devices_common_only:
        return 'Solo Comuni'
    elif machine_type in laser_devices:
        return 'Laser'
    elif machine_type in hydraulic_devices:
        return 'Idraulico'
    elif machine_type in coolant_devices:
        return 'Refrigerante'
    elif machine_type in heat_devices:
        return 'Calore'
    else:
        return 'Altro'

df['Device_Category'] = df['Machine_Type'].apply(classify_device)

print("=== CLASSIFICAZIONE DISPOSITIVI ===")
category_counts = df['Device_Category'].value_counts()
display(category_counts)

# Visualizzazione
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='Device_Category', order=category_counts.index)
plt.title('Distribuzione Categorie di Dispositivi')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 3. Analisi delle Variabili Target

In [ ]:
# Analisi della variabile target binaria
print("=== ANALISI FAILURE_WITHIN_7_DAYS ===")
failure_dist = df['Failure_Within_7_Days'].value_counts()
failure_percent = df['Failure_Within_7_Days'].value_counts(normalize=True) * 100

print("Distribuzione:")
for val, count, perc in zip(failure_dist.index, failure_dist.values, failure_percent.values):
    print(f"  {val}: {count} ({perc:.2f}%)")

# Analisi della vita rimanente
print("\n=== ANALISI REMAINING_USEFUL_LIFE_DAYS ===")
life_stats = df['Remaining_Useful_Life_days'].describe()
display(life_stats)

# Visualizzazioni
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Distribuzione failure entro 7 giorni
axes[0,0].pie(failure_dist.values, labels=['No Failure', 'Failure'], autopct='%1.1f%%', startangle=90)
axes[0,0].set_title('Distribuzione Guasti Entro 7 Giorni')

# Distribuzione vita rimanente
axes[0,1].hist(df['Remaining_Useful_Life_days'], bins=50, edgecolor='black', alpha=0.7)
axes[0,1].set_title('Distribuzione Vita Rimanente (giorni)')
axes[0,1].set_xlabel('Giorni')
axes[0,1].set_ylabel('Frequenza')

# Boxplot vita rimanente per categoria failure
sns.boxplot(data=df, x='Failure_Within_7_Days', y='Remaining_Useful_Life_days', ax=axes[1,0])
axes[1,0].set_title('Vita Rimanente vs Failure Entro 7 Giorni')

# Distribuzione vita rimanente per categoria dispositivo
df.groupby('Device_Category')['Remaining_Useful_Life_days'].mean().plot(kind='bar', ax=axes[1,1])
axes[1,1].set_title('Vita Rimanente Media per Categoria Dispositivo')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 4. Analisi delle Features Comuni

In [ ]:
# Statistiche descrittive delle features comuni
common_features = ['Installation_Year', 'Operational_Hours', 'Temperature_C', 'Vibration_mms', 'Sound_dB', 'Oil_Level_pct', 'Coolant_Level_pct', 'Power_Consumption_kW', 'Last_Maintenance_Days_Ago', 'Maintenance_History_Count', 'Failure_History_Count', 'AI_Supervision', 'Error_Codes_Last_30_Days', 'AI_Override_Events']
print("=== STATISTICHE FEATURES COMUNI ===")
common_stats = df[common_features].describe()
display(common_stats)

In [ ]:
# Visualizzazione distribuzioni features comuni
n_cols = 4
n_rows = (len(common_features) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5*n_rows))
axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes

for i, feature in enumerate(common_features):
    if i < len(axes):
        axes[i].hist(df[feature].dropna(), bins=30, edgecolor='black', alpha=0.7)
        axes[i].set_title(f'Distribuzione {feature}')
        axes[i].set_xlabel(feature)
        axes[i].set_ylabel('Frequenza')

# Nascondere assi non utilizzati
for i in range(len(common_features), len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Matrice di correlazione features comuni
print("=== MATRICE DI CORRELAZIONE FEATURES COMUNI ===")
correlation_matrix = df[common_features].corr()

plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, fmt='.2f', cbar_kws={"shrink": .8})
plt.title('Matrice di Correlazione - Features Comuni')
plt.tight_layout()
plt.show()

# Correlazioni più forti
correlation_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_val = correlation_matrix.iloc[i, j]
        if abs(corr_val) > 0.5:
            correlation_pairs.append({
                'Feature_1': correlation_matrix.columns[i],
                'Feature_2': correlation_matrix.columns[j],
                'Correlazione': corr_val
            })

if correlation_pairs:
    print("\nCorrelazioni forti (|r| > 0.5):")
    corr_df = pd.DataFrame(correlation_pairs).sort_values('Correlazione', key=abs, ascending=False)
    display(corr_df)
else:
    print("\nNessuna correlazione forte trovata tra le features comuni.")

## 5. Analisi Features Aggiuntive

In [ ]:
# Analisi features aggiuntive
additional_features = ['Laser_Intensity', 'Hydraulic_Pressure_bar', 'Coolant_Flow_L_min', 'Heat_Index']

print("=== ANALISI FEATURES AGGIUNTIVE ===")
for feature in additional_features:
    if feature in df.columns:
        non_null_count = df[feature].notna().sum()
        null_count = df[feature].isna().sum()
        print(f"\n{feature}:")
        print(f"  Valori non nulli: {non_null_count} ({non_null_count/len(df)*100:.1f}%)")
        print(f"  Valori nulli: {null_count} ({null_count/len(df)*100:.1f}%)")
        
        if non_null_count > 0:
            stats = df[feature].describe()
            print(f"  Media: {stats['mean']:.2f}")
            print(f"  Deviazione standard: {stats['std']:.2f}")
            print(f"  Min: {stats['min']:.2f}")
            print(f"  Max: {stats['max']:.2f}")

# Visualizzazione features aggiuntive
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for i, feature in enumerate(additional_features):
    if feature in df.columns and i < 4:
        feature_data = df[feature].dropna()
        if len(feature_data) > 0:
            axes[i].hist(feature_data, bins=30, edgecolor='black', alpha=0.7)
            axes[i].set_title(f'Distribuzione {feature}')
            axes[i].set_xlabel(feature)
            axes[i].set_ylabel('Frequenza')
        else:
            axes[i].text(0.5, 0.5, f'Nessun dato\nper {feature}', 
                        ha='center', va='center', transform=axes[i].transAxes)
            axes[i].set_title(f'{feature} - Nessun Dato')

plt.tight_layout()
plt.show()

## 6. Analisi degli Outliers

In [ ]:
# Identificazione outliers usando il metodo IQR
def identify_outliers(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

print("=== ANALISI OUTLIERS (METODO IQR) ===")
outlier_summary = []

numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
# Rimuovere le colonne ID e categoriche
numeric_features = [f for f in numeric_features if f not in ['Machine_ID', 'Failure_Within_7_Days']]

for feature in numeric_features[:10]:  # Analizzare prime 10 features
    outliers, lower, upper = identify_outliers(df, feature)
    outlier_count = len(outliers)
    outlier_percent = (outlier_count / len(df)) * 100
    
    outlier_summary.append({
        'Feature': feature,
        'Outliers': outlier_count,
        'Percentuale': outlier_percent,
        'Lower_Bound': lower,
        'Upper_Bound': upper
    })

outlier_df = pd.DataFrame(outlier_summary).sort_values('Percentuale', ascending=False)
display(outlier_df)

# Visualizzazione boxplot per features con più outliers
top_outlier_features = outlier_df.head(6)['Feature'].tolist()

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, feature in enumerate(top_outlier_features):
    sns.boxplot(data=df, y=feature, ax=axes[i])
    axes[i].set_title(f'Boxplot {feature}')

plt.tight_layout()
plt.show()

## 7. Analisi delle Relazioni con il Target

In [ ]:
# Correlazione features numeriche con il target
print("=== CORRELAZIONE CON IL TARGET (FAILURE_WITHIN_7_DAYS) ===")

target_correlations = []
for feature in numeric_features:
    corr = df[feature].corr(df['Failure_Within_7_Days'])
    if not np.isnan(corr):
        target_correlations.append({
            'Feature': feature,
            'Correlazione': corr
        })

target_corr_df = pd.DataFrame(target_correlations).sort_values('Correlazione', key=abs, ascending=False)
display(target_corr_df.head(10))

# Visualizzazione correlazioni
plt.figure(figsize=(12, 8))
top_features = target_corr_df.head(10)
colors = ['red' if x < 0 else 'green' for x in top_features['Correlazione']]
plt.barh(top_features['Feature'], top_features['Correlazione'], color=colors, alpha=0.7)
plt.title('Top 10 Correlazioni con Failure_Within_7_Days')
plt.xlabel('Correlazione')
plt.axvline(x=0, color='black', linestyle='-', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# Analisi differenze tra gruppi (failure vs no failure)
print("=== CONFRONTO GRUPPI: FAILURE VS NO FAILURE ===")

failure_group = df[df['Failure_Within_7_Days'] == 1]
no_failure_group = df[df['Failure_Within_7_Days'] == 0]

# Confronto statistiche
comparison_results = []
for feature in target_corr_df.head(8)['Feature']:  # Top 8 features
    failure_mean = failure_group[feature].mean()
    no_failure_mean = no_failure_group[feature].mean()
    
    # Test t per differenza delle medie
    from scipy.stats import ttest_ind
    t_stat, p_value = ttest_ind(failure_group[feature].dropna(), 
                               no_failure_group[feature].dropna())
    
    comparison_results.append({
        'Feature': feature,
        'Failure_Mean': failure_mean,
        'No_Failure_Mean': no_failure_mean,
        'Differenza': failure_mean - no_failure_mean,
        'P_Value': p_value
    })

comparison_df = pd.DataFrame(comparison_results)
display(comparison_df)

# Visualizzazione confronti
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for i, feature in enumerate(comparison_df['Feature']):
    data_to_plot = [no_failure_group[feature].dropna(), failure_group[feature].dropna()]
    axes[i].boxplot(data_to_plot, labels=['No Failure', 'Failure'])
    axes[i].set_title(f'{feature}')
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 8. Conclusioni e Insights

In [ ]:
print("=== RIEPILOGO ANALISI ESPLORATIVA ===")
print(f"\n1. STRUTTURA DATASET:")
print(f"   - Totale record: {len(df):,}")
print(f"   - Totale features: {len(df.columns)}")
print(f"   - Tipi di dispositivi: {df['Machine_Type'].nunique()}")
print(f"   - Memoria utilizzata: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print(f"\n2. DISTRIBUZIONE TARGET:")
failure_rate = df['Failure_Within_7_Days'].mean() * 100
print(f"   - Tasso di failure entro 7 giorni: {failure_rate:.2f}%")
print(f"   - Vita rimanente media: {df['Remaining_Useful_Life_days'].mean():.1f} giorni")
print(f"   - Vita rimanente mediana: {df['Remaining_Useful_Life_days'].median():.1f} giorni")

print(f"\n3. QUALITÀ DATI:")
total_missing = df.isnull().sum().sum()
missing_percent = (total_missing / (len(df) * len(df.columns))) * 100
print(f"   - Valori mancanti totali: {total_missing} ({missing_percent:.2f}%)")

print(f"\n4. FEATURES PIÙ CORRELATE CON IL TARGET:")
for i, row in target_corr_df.head(5).iterrows():
    print(f"   - {row['Feature']}: {row['Correlazione']:.3f}")

print(f"\n5. RACCOMANDAZIONI PER IL PREPROCESSING:")
print(f"   - Il dataset sembra ben strutturato con pochi valori mancanti")
print(f"   - Presenza di outliers da gestire in fase di preprocessing")
print(f"   - Features aggiuntive hanno molti valori nulli (da gestire per tipo dispositivo)")
print(f"   - Il target è moderatamente bilanciato ({failure_rate:.1f}% failure rate)")
print(f"   - Alcune features mostrano correlazioni significative con il target")

# Salvataggio risultati esplorazione
exploration_results = {
    'dataset_shape': df.shape,
    'failure_rate': failure_rate,
    'missing_data_percent': missing_percent,
    'top_correlated_features': target_corr_df.head(10).to_dict('records'),
    'device_categories': df['Device_Category'].value_counts().to_dict(),
    'outlier_summary': outlier_df.to_dict('records')
}

import json
os.makedirs('data/processed', exist_ok=True)with open('data/processed/exploration_results.json', 'w') as f:
    json.dump(exploration_results, f, indent=2, default=str)

print(f"\n✅ Risultati dell'esplorazione salvati in 'exploration_results.json'")